Importing packages

In [82]:
import requests
from bs4 import BeautifulSoup
from mysql.connector import connect
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

define the function for save date to our database

In [36]:
def save_to_database(car_list):
    cnx = connect(user= 'root', password='44527077', host='127.0.0.1', database='advance_python')
    cursor = cnx.cursor()
#     cursor.execute('CREATE TABLE car_db (price VARCHAR(30), milage VARCHAR(40), year INT)')
#     cnx.commit()
    for item in car_list:
        cursor.execute(f'INSERT INTO car_db VALUE (\'{item[0]}\', \'{item[1]}\', {item[2]})')
        cnx.commit()
    print('Done!')
    cnx.close()

define the function for read data from database

In [42]:
def read_from_database():
    cnx = connect(user= 'root', password='44527077', host='127.0.0.1', database='advance_python')
    cursor = cnx.cursor()
    cursor.execute('SELECT * FROM car_db')
    car_milages = []
    car_years = []
    car_prices = []
    for price, milage, year in cursor:
        car_milages.append(milage)
        car_prices.append(price)
        car_years.append(year)

    print('Reading Done!')
    cnx.close()
    return dict(year= car_years, milage= car_milages, price= car_prices)

define the function for getting information that we want from website and internet

In [26]:
def getting_data(model, page):
    car_listings = []

    for page in range(1, 11):
        url = f'https://www.truecar.com/used-cars-for-sale/listings/{model}/?page={page}'
        request = requests.get(url)
        soup = BeautifulSoup(request.text, 'html')
        
        listings = soup.find_all('div', class_='card-content')
        
        for listing in listings:        
            price = listing.find('span', attrs={'data-test': 'vehicleListingPriceAmount'}).text.strip()
            mileage = listing.find('div', class_='truncate text-xs', attrs={'data-test': 'vehicleMileage'}).text.strip()
            year = listing.find('span', class_="vehicle-card-year text-xs").text.strip()
            car_listings.append((price, mileage, year))
    return car_listings


I want to read 10 page of bmw model from truecar website

In [27]:
car_list = getting_data('bmw', 10)

In [28]:
len(car_list)

309

In [29]:
car_list[:5]

[('$26,641', '51,632 miles', '2020'),
 ('$19,995', '57,505 miles', '2017'),
 ('$39,000', '26,352 miles', '2022'),
 ('$19,490', '108,231 miles', '2020'),
 ('$24,998', '54,135 miles', '2020')]

save the information to our database

In [37]:
save_to_database(car_list)

Done!


read data from database and transform to dataframe

In [44]:
data = read_from_database()

Reading Done!


In [55]:
df = pd.DataFrame(data)
df.head()

,year,milage,price
0,2020,"51,632 miles","$26,641"
1,2017,"57,505 miles","$19,995"
2,2022,"26,352 miles","$39,000"
3,2020,"108,231 miles","$19,490"
4,2020,"54,135 miles","$24,998"


save dataframe into a csv file

In [47]:
df.to_csv('data.csv')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    309 non-null    int64 
 1   milage  309 non-null    object
 2   price   309 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.4+ KB


In [61]:
df.milage

0       51,632 miles
1       57,505 miles
2       26,352 miles
3      108,231 miles
4       54,135 miles
           ...      
304      9,412 miles
305     43,682 miles
306     52,036 miles
307     22,096 miles
308      4,262 miles
Name: milage, Length: 309, dtype: object

preprocessing

In [64]:
df.milage = df.milage.str.extract('(.+)\s',expand=False)

In [66]:
df.milage = df.milage.str.replace(',', '')

In [67]:
df.milage = df.milage.astype('int')

In [68]:
df.head()

,year,milage,price
0,2020,51632,"$26,641"
1,2017,57505,"$19,995"
2,2022,26352,"$39,000"
3,2020,108231,"$19,490"
4,2020,54135,"$24,998"


In [70]:
df.price = df.price.str.extract('\$(.+)',expand=False)
df.price = df.price.str.replace(',', '')
df.price = df.price.astype('int')

In [71]:
df.head()

,year,milage,price
0,2020,51632,26641
1,2017,57505,19995
2,2022,26352,39000
3,2020,108231,19490
4,2020,54135,24998


EDA

In [75]:
df.groupby('year').agg('mean')

,milage,price
year,,
1997,52000.000000,9999.000000
2005,138321.000000,5991.000000
2007,83077.500000,14557.250000
2008,123963.500000,10895.000000
2009,103586.250000,9123.250000
2010,133312.500000,10249.000000
2011,102606.812500,10438.937500
2012,115800.000000,11194.428571
2013,96466.333333,16915.047619


In [77]:
df.groupby('year')['milage'].count()

year
1997     1
2005     1
2007     4
2008     2
2009     4
2010     2
2011    16
2012     7
2013    21
2014    22
2015    19
2016    19
2017    17
2018     9
2019    23
2020    81
2021    49
2022     8
2023     4
Name: milage, dtype: int64

Train and Test split

In [78]:
X = df[['year','milage']].values
Y = df.price.values

In [79]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, train_size=0.7)

In [81]:
print('size of train x : ', train_x.shape, ' size of train y : ', train_y.shape)
print('size of test x : ', test_x.shape, ' size of test y : ', test_y.shape)

size of train x :  (216, 2)  size of train y :  (216,)
size of test x :  (93, 2)  size of test y :  (93,)


Standarding the data

In [84]:
std = StandardScaler()
x_train = std.fit_transform(train_x)
x_test = std.transform(test_x)

train the model

In [85]:
model = LinearRegression()
model.fit(x_train, train_y)

LinearRegression()

predict with model

In [86]:
predict_train = model.predict(x_train)
predict_test = model.predict(x_test)

In [88]:
from sklearn.metrics import r2_score

print(f'r2 score is train : {r2_score(train_y, predict_train)}')
print(f'r2 score is test : {r2_score(test_y, predict_test)}')

r2 score is train : 0.6121416134818954
r2 score is test : 0.3983133435678775


Mohammad Rasoul Azizi
    (Summer of 2023)